In [1]:
import openai
import json
from tqdm import tqdm


openai.api_key = ""


def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_prompt(message: str):
    return f"""
    Identify the following item from the telegram chat message, written in Russian: 
    - What product is the user interested in
    
    The message is delimited with triple backticks. \
    Format your response as a JSON object with \
    "products" as the keys. 
    If the information isn't present, use "" \
    as the value.
  
    Message: '''{message}'''
    """

In [2]:
with open('channel_messages.json', 'r', encoding="utf8") as f:
    messages = json.load(f)

In [7]:
products = []
for message in tqdm(messages):
    prompt = get_prompt(message)
    gpt_answer = json.loads(get_completion(prompt))
    if type(gpt_answer.get('products', [])) == str:
        if gpt_answer.get('products', []).strip() == '':
              continue
        else:
            gpt_answer['products'] = [gpt_answer.get('products')]
    
    if len(gpt_answer.get('products', [])) < 1:
        continue
    else:
        products.append({
            'date': message.get('date', ''),
            'user_id': message.get('user_id', ''),
            "message_id": message.get('message_id', ''),
            'products': gpt_answer.get('products', ''),
        })
with open('product_requests.json', 'w', encoding='utf8') as outfile:
		json.dump(products, outfile, ensure_ascii=False)

100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


In [6]:
#message = """Ребята, не знаете где купить лопату для выпечки пиццы и соевое молоко Alpro?"""
#message = """Всем привет! Подскажите по овсяному молоку, пожалуйста 🙏🏻 покупала Nемолоко в фикс прайсе пол года, а теперь его там нет😢 
#
#Можно ли его где-то в Тбилиси купить?"""
#message = """Котики, а есть где в Тбилиси купить пищевую глину? 
#Не мне, подруга спрашивает"""
#message ="""Привет!
#
#Пока что меня зовут Бабайка. Сейчас мне примерно 1,5месяца. Мама-кошка бросила меня, и я провела несколько страшных дней и ночей в водостоке. Я много кричала и плакала, засыпала от бессилия, но потом все повторялось вновь.
#Сейчас меня забрали к себе Люди. Они научили меня есть из мисочки и ходить по делам в лоточек. А еще они научили меня, что Человек и Кошка могут подарить друг другу много тепла. Теперь я хорошо кушаю паштеты и паучи для котят, сладко сплю и очень много играю. Я люблю находиться рядом с людьми, но мне не скучно и одной, ведь я любознательная и самостоятельная.
#Я очень хочу найти любящего Человека и быть с ним одной семьей. Я подарю много ласки и любви взамен на доброту. 
#
#Я нахожусь в Тбилиси. И смогу приехать к вам по городу, если вы разглядели во мне свое будущее любимое существо. 
#Вы можете спросить про меня по телефону: +995 558 514 022 
#Или написать в личные сообщения"""
#message = """Всем привет, кто знает где сейчас купить светлый мисо, в сакуре нет"""
message = """Добрый вечер 

Завтра будет официальный ответ от магазина 

Команда ветеринаров на нашей стороне 

Будут комментарии по всем пунктам поста"""
prompt = get_prompt(message)

In [7]:
get_completion(prompt)

'{"products": ""}'